In [ ]:
#r "nuget: Microsoft.DotNet.Interactive.SqlServer, *-*"

In [ ]:
#!connect mssql --kernel-name sql2025 --connection-string "Server=SQL2025;TrustServerCertificate=True;Integrated Security=True"

In [ ]:
SELECT @@VERSION

In [ ]:
USE master
IF EXISTS (SELECT name FROM sys.databases WHERE name='LinkedInCourseDB')BEGIN
    ALTER DATABASE LinkedInCourseDB SET SINGLE_USER WITH ROLLBACK IMMEDIATE
    DROP DATABASE LinkedInCourseDB
END
GO
CREATE DATABASE LinkedInCourseDB
GO
USE LinkedInCourseDB


In [ ]:
create master key encryption by password = 'MyTest!Mast3rP4ss'

In [ ]:
sp_configure 'external rest endpoint enabled', 1
RECONFIGURE WITH OVERRIDE

In [ ]:
DECLARE @response NVARCHAR(max)

EXEC Sp_invoke_external_rest_endpoint
  @url = N'https://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL?format=json&date=2022',
  @method = 'GET',
  @response = @response output

SELECT @response

In [ ]:
DECLARE @response NVARCHAR(max)

EXEC Sp_invoke_external_rest_endpoint
  @url = N'https://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL?format=json&date=2022',
  @method = 'GET',
  @response = @response output

SELECT TOP 5 [Value] FROM Openjson(Json_query(@response, '$.result[1]')) A 

In [ ]:
DECLARE @response NVARCHAR(max)

EXEC Sp_invoke_external_rest_endpoint
  @url = N'https://reqres.in/api/products/',
  @headers = '{"x-api-key":"reqres-free-v1"}',
  @method = 'GET',
  @response = @response output

SELECT * FROM   Openjson(Json_query(@response, '$.result.data')) A 

In [ ]:
CREATE DATABASE SCOPED CREDENTIAL [https://reqres.in/api/]
WITH IDENTITY = 'HTTPEndpointHeaders', SECRET = '{"x-api-key":"reqres-free-v1"}'

In [ ]:
DECLARE @response NVARCHAR(max)

EXEC Sp_invoke_external_rest_endpoint
  @url = N'https://reqres.in/api/products/',
  @credential = [https://reqres.in/api/],
  @method = 'GET',
  @response = @response output

SELECT * FROM   Openjson(Json_query(@response, '$.result.data')) A 

In [ ]:
DECLARE @response NVARCHAR(max)

EXEC Sp_invoke_external_rest_endpoint
  @url = N'https://reqres.in/api/products/',
  @credential = [https://reqres.in/api/],
  @method = 'GET',
  @response = @response output

SELECT Json_value(value, '$.name')          name,
       Json_value(value, '$.color')         color,
       Json_value(value, '$.pantone_value') pantone_value
FROM   Openjson(Json_query(@response, '$.result.data'), 'strict $') AS export 

IDENTITY could also be HttpEndpointQueryString, MANAGED IDENTITY, a user, a SAS token or OAUTH.

In [ ]:
IF EXISTS (SELECT * FROM sys.database_scoped_credentials WHERE NAME = 'https://management.azure.com/')
  BEGIN
      DROP DATABASE scoped credential [https://management.azure.com/]
  END

GO

DECLARE @response NVARCHAR(max)

EXEC Sp_invoke_external_rest_endpoint
  @url = N'https://login.microsoftonline.com/XXX/oauth2/token',
  @headers='{"content-type":"application/x-www-form-urlencoded"}',
  @payload=N'grant_type=client_credentials&client_id=XXX&client_secret=XXX&resource=https%3A%2F%2Fmanagement.azure.com%2F',
  @method = 'POST',
  @response = @response output

DECLARE @secret NVARCHAR(max)

SELECT @secret = value
FROM   Openjson(Json_query(@response, '$.result')) A
WHERE  [key] = 'access_token'

SET @secret = N'{"Authorization":"Bearer ' + @secret + '"}'
SET @secret = N'CREATE DATABASE SCOPED CREDENTIAL [https://management.azure.com/] WITH IDENTITY = ''HTTPEndpointHeaders'', SECRET = ''' + @secret + ''''

EXEC Sp_executesql  @secret 

In [ ]:
DECLARE @response NVARCHAR(max)

EXEC Sp_invoke_external_rest_endpoint @url = 'https://management.azure.com/subscriptions/92cc49b9-95ca-4935-86e1-1545d29bc50a/providers/Microsoft.Compute/disks?api-version=2025-01-02',
@credential = [https://management.azure.com/],
@method = 'GET',
@response = @response output

SELECT *
FROM   (SELECT Json_value(value, '$.name')            DiskName,
               Json_value(value, '$.properties.tier') DiskTier
        FROM   Openjson(Json_query(@response, '$.result.value'))) A

In [ ]:
DECLARE @response NVARCHAR(max)
DECLARE @Result int

EXEC @Result = Sp_invoke_external_rest_endpoint
  @url = N'https://reqres.in/api/products/',
  @credential = [https://reqres.in/api/],
  @method = 'GET',
  @response = @response output

SELECT @Result,@Response

In [ ]:
DECLARE @response NVARCHAR(max)
DECLARE @Result int

EXEC @Result = Sp_invoke_external_rest_endpoint
  @url = N'https://api.exchangeratesapi.io/v1/latest',
  @method = 'GET',
  @response = @response output

SELECT @Result,@Response

In [ ]:
DECLARE @response NVARCHAR(max)


EXEC Sp_invoke_external_rest_endpoint
  @url = N'https://openlibrary.org/search.json?q=sql+server+2022',
  @timeout = 230,
  @method = 'GET',
  @response = @response output

SELECT * FROM   Openjson(Json_query(@response, '$.result')) A

In [ ]:
DECLARE @response NVARCHAR(max)


EXEC Sp_invoke_external_rest_endpoint
  @url = N'https://openlibrary.org/search.json?q=sql+server+2022',
  @timeout = 230,
  @method = 'GET',
  @response = @response output

SELECT top 5 json_value(value, '$.key') OL_key,json_value(value, '$.author_name[0]') MainAuthor,Json_value(value, '$.title') Title FROM   Openjson(Json_query(@response, '$.result.docs')) A


In [ ]:
IF  EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'[dbo].[SQLBooks]') AND type in (N'U'))
DROP TABLE [dbo].[SQLBooks]
GO
CREATE TABLE [dbo].[SQLBooks](
	[ID] [int] IDENTITY(1,1) NOT NULL,
	[OL_key] [nvarchar](100) NULL,
	[MainAuthor] [nvarchar](250) NULL,
	[Title] [nvarchar](1000) NULL
) ON [PRIMARY] 

In [ ]:
DECLARE @response NVARCHAR(max)

EXEC Sp_invoke_external_rest_endpoint
  @url = N'https://openlibrary.org/search.json?q=sql+server+2022',
  @timeout = 230,
  @method = 'GET',
  @response = @response output

TRUNCATE TABLE SQLBooks
INSERT INTO SQLBooks (OL_Key,MainAuthor,Title)
SELECT json_value(value, '$.key') OL_key,json_value(value, '$.author_name[0]') MainAuthor,Json_value(value, '$.title') Title 
FROM   Openjson(Json_query(@response, '$.result.docs')) A


In [ ]:
SELECT * FROM SQLBooks

In [ ]:
DECLARE @response NVARCHAR(max)

EXEC Sp_invoke_external_rest_endpoint
  @url = N'https://openlibrary.org/search.json?q=sql+server',
  @timeout = 230,
  @method = 'GET',
  @response = @response output

SELECT * FROM   Openjson(Json_query(@response, '$.result')) A where [key] = 'numFound'

TRUNCATE TABLE SQLBooks
INSERT INTO SQLBooks (OL_Key,MainAuthor,Title)
SELECT json_value(value, '$.key') OL_key,json_value(value, '$.author_name[0]') MainAuthor,Json_value(value, '$.title') Title 
FROM   Openjson(Json_query(@response, '$.result.docs')) A

In [ ]:
TRUNCATE TABLE SQLBooks
SET NOCOUNT ON

DECLARE @response NVARCHAR(MAX),
        @url NVARCHAR(MAX),
        @start INT = 0,
        @hasMore BIT = 1

WHILE @hasMore = 1
BEGIN
    SET @url = 'https://openlibrary.org/search.json?q=sql+server&limit=100&offset=' + CAST(@start AS NVARCHAR)

    EXEC Sp_invoke_external_rest_endpoint
        @url = @url,
        @method = 'GET',
        @response = @response OUTPUT

    IF @response IS NULL OR LEN(@response) < 50
    BEGIN
        SET @hasMore = 0
        BREAK
    END

    INSERT INTO SQLBooks (OL_Key, MainAuthor,  Title)
    SELECT
        JSON_VALUE(value, '$.key') OL_Key,
        ISNULL(JSON_VALUE(value, '$.author_name[0]'),'') MainAuthor,
        JSON_VALUE(value, '$.title') Title
    FROM OPENJSON(JSON_QUERY(@response, '$.result.docs'))

    IF (SELECT COUNT(*) FROM OPENJSON(JSON_QUERY(@response, '$.result.docs'))) < 100
        SET @hasMore = 0
    ELSE
        SET @start += 100

END
SELECT COUNT(*) FROM SQLBooks

In [ ]:
DECLARE @response NVARCHAR(max)
DECLARE @prompt NVARCHAR(max) = N'what is the airspeed velocity of an unladen swallow?'
DECLARE @model NVARCHAR(250) = N'mistral'
DECLARE @payload NVARCHAR(max) = N'{"model":"' + @model + '","prompt":"' + @prompt + '","stream": false }'

EXEC Sp_invoke_external_rest_endpoint
  @url = N'https://ai-gpu.lab.bwdemo.io:443/api/generate',
  @payload = @payload,
  @timeout = 230,
  @response = @response output

SELECT value
FROM   Openjson(Json_query(@response, '$.result')) A
WHERE  [key] = 'response' 

In [ ]:
SELECT TOP 1 MainAuthor,Title FROM SQLBooks WHERE Title like '%2022%' order by MainAuthor

In [ ]:
SELECT TOP 1 'What can you tell me about the book ''' + Title + ''' by ''' + MainAuthor + '''?' FROM SQLBooks WHERE Title like '%2022%' order by MainAuthor 

In [ ]:
DECLARE @response NVARCHAR(max)
DECLARE @prompt NVARCHAR(max) 
SELECT TOP 1 @prompt='What can you tell me about the book ''' + Title + ''' by ''' + MainAuthor + '''?' FROM SQLBooks WHERE Title like '%2022%' order by MainAuthor 
DECLARE @model NVARCHAR(250) = N'mistral'
DECLARE @payload NVARCHAR(max) = N'{"model":"' + @model + '","prompt":"' + @prompt + '","stream": false }'

EXEC Sp_invoke_external_rest_endpoint
  @url = N'https://ai-gpu.lab.bwdemo.io:443/api/generate',
  @payload = @payload,
  @timeout = 230,
  @response = @response output

SELECT value
FROM   Openjson(Json_query(@response, '$.result')) A
WHERE  [key] = 'response' 